In [1]:
import sympy as sp
import numpy as np
import re
from tensorflow import keras
from keras.layers import Input, Dense, Embedding, MultiHeadAttention, LayerNormalization, Dropout, LSTM
from keras.models import Model
from keras.callbacks import EarlyStopping

### 1. Generate Taylor Series Expansions ###

In [2]:
def generate_taylor_expansions(functions, order=4):
    # Generates Taylor expansions for given functions up to the specified order.
    taylor_expansions = {}
    x = sp.symbols('x')
    for func in functions:
        try:
            expansion = sp.series(func, x, 0, order + 1).removeO()
            taylor_expansions[str(func)] = expansion
        except Exception as e:
            print(f"Error generating expansion for {func}: {e}")
    return taylor_expansions

### 2. Tokenization Function ###

In [3]:
def tokenize(expression):
    # Tokenizes a mathematical expression using regex.
    tokens = re.findall(r'\S+', expression)
    return tokens

# Define symbolic functions
x = sp.symbols('x')
functions = [sp.sin(x), sp.cos(x), sp.exp(x), sp.log(1 + x)]

# Compute Taylor expansions and tokenize them
taylor_expansions = generate_taylor_expansions(functions)
tokenized_data = {func: tokenize(str(exp)) for func, exp in taylor_expansions.items()}

# Vocabulary creation
unique_tokens = set(token for tokens in tokenized_data.values() for token in tokens)
token_to_index = {token: i for i, token in enumerate(unique_tokens)}

# Convert tokenized expressions into numerical sequences
numerical_sequences = [[token_to_index[token] for token in tokens] for tokens in tokenized_data.values()]
max_length = max(len(seq) for seq in numerical_sequences)

# Padding sequences to max_length
X_train = np.array([seq + [0] * (max_length - len(seq)) for seq in numerical_sequences])
y_train = X_train.copy()

vocab_size = len(unique_tokens)
embedding_dim = 64

### 3. Build Hybrid LSTM + Transformer Model ###

In [61]:
def build_hybrid_model(vocab_size, embedding_dim, max_length):
    input_layer = Input(shape=(max_length,))
    embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)

    # LSTM Layer
    lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
    lstm_layer = Dropout(0.2)(lstm_layer)
    lstm_layer = LSTM(64, return_sequences=True)(lstm_layer)

    # Transformer (Multi-Head Attention)
    attention_layer = MultiHeadAttention(num_heads=4, key_dim=embedding_dim)(lstm_layer, lstm_layer)
    attention_layer = Dropout(0.2)(attention_layer)
    normalized_layer = LayerNormalization(epsilon=1e-6)(attention_layer + lstm_layer)

    # Output Layer
    output_layer = Dense(vocab_size, activation='softmax')(normalized_layer)

    # Build and Compile Model
    hybrid_model = Model(inputs=input_layer, outputs=output_layer)
    hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return hybrid_model

# Create and Train the Hybrid Model
hybrid_model = build_hybrid_model(vocab_size, embedding_dim, max_length)
hybrid_model.summary()

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
hybrid_model.fit(X_train, y_train, epochs=10, batch_size=4, validation_split=0.2, callbacks=[early_stopping])

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)    │ (None, 9)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_8 (Embedding)       │ (None, 9, 64)             │             640 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_16 (LSTM)                │ (None, 9, 128)            │          98,816 │ embedding_8[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_24 (Dropout)          │ (None, 9, 128)            │               0 │ lstm_16[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_17 (LSTM)                │ (None, 9, 64)             │          49,408 │ dropout_24[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_8        │ (None, 9, 64)             │          66,368 │ lstm_17[0][0],             │
│ (MultiHeadAttention)          │                           │                 │ lstm_17[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_26 (Dropout)          │ (None, 9, 64)             │               0 │ multi_head_attention_8[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_8 (Add)                   │ (None, 9, 64)             │               0 │ dropout_26[0][0],          │
│                               │                           │                 │ lstm_17[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_8         │ (None, 9, 64)             │             128 │ add_8[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 9, 10)             │             650 │ layer_normalization_8[0][… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 216,010 (843.79 KB)

 Trainable params: 216,010 (843.79 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.0000e+00 - loss: 3.4354WARNING:tensorflow:5 out of the last 23 calls to <function TensorFlowTrainer._make_function.<locals>.multi_step_on_iterator at 0x0000027B5238D4E0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0000e+00 - loss: 3.4354 - val_accuracy: 0.3333 - val_loss: 3.0748
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.5185 - loss: 2.6954 - 